# Anti-Rectifier

The example demonstrates how to write custom layers for Keras.

We build a custom activation layer called 'Antirectifier',
which modifies the shape of the tensor that passes through it.

We need to specify two methods: `compute_output_shape` and `call`.

Note that the same result can also be achieved via a Lambda layer.

Because our custom layer is written with primitives from the Keras
backend (`K`), our code can run both on TensorFlow and Theano.

In [1]:
import keras
from keras.models import Sequential
from keras.layers import Layer, Dense, Dropout, Activation
from keras import backend as K

from keras.datasets import mnist

Using TensorFlow backend.


In [2]:
class Antirectifier(Layer):
    '''This is the combination of a sample-wise
    L2 normalization with the concatenation of the
    positive part of the input with the negative part
    of the input. The result is a tensor of samples that are
    twice as large as the input samples.

    It can be used in place of a ReLU.

    # Input shape
        2D tensor of shape (samples, n)

    # Output shape
        2D tensor of shape (samples, 2*n)

    # Theoretical justification
        When applying ReLU, assuming that the distribution
        of the previous output is approximately centered around 0.,
        you are discarding half of your input. This is inefficient.

        Antirectifier allows to return all-positive outputs like ReLU,
        without discarding any data.

        Tests on MNIST show that Antirectifier allows to train networks
        with twice less parameters yet with comparable
        classification accuracy as an equivalent ReLU-based network.
    '''

    def compute_output_shape(self, input_shape):
        shape = list(input_shape)
        assert len(shape) == 2  # only valid for 2D tensors
        shape[-1] *= 2
        return tuple(shape)

    def call(self, inputs):
        inputs -= K.mean(inputs, axis=1, keepdims=True)
        inputs = K.l2_normalize(inputs, axis=1)
        pos = K.relu(inputs)
        neg = K.relu(-inputs)
        return K.concatenate([pos, neg], axis=1)

In [3]:
# global parameters
batch_size = 128
num_classes = 10
epochs = 40

In [4]:
# the data, split between train and test sets
(x_train, y_train), (x_test, y_test) = mnist.load_data()

In [5]:
x_train = x_train.reshape(60000, 784)
x_test = x_test.reshape(10000, 784)
x_train = x_train.astype('float32')
x_test = x_test.astype('float32')
x_train /= 255
x_test /= 255
print(x_train.shape[0], 'train samples')
print(x_test.shape[0], 'test samples')

60000 train samples
10000 test samples


In [6]:
# convert class vectors to binary class matrices
y_train = keras.utils.to_categorical(y_train, num_classes)
y_test = keras.utils.to_categorical(y_test, num_classes)

In [7]:
# build the model
model = Sequential()

model.add(Dense(256, input_shape=(784,)))
model.add(Antirectifier())
model.add(Dropout(0.1))

model.add(Dense(256))
model.add(Antirectifier())
model.add(Dropout(0.1))

model.add(Dense(num_classes))
model.add(Activation('softmax'))

In [8]:
# compile the model
model.compile(loss='categorical_crossentropy',
              optimizer='rmsprop',
              metrics=['accuracy'])
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_1 (Dense)              (None, 256)               200960    
_________________________________________________________________
antirectifier_1 (Antirectifi (None, 512)               0         
_________________________________________________________________
dropout_1 (Dropout)          (None, 512)               0         
_________________________________________________________________
dense_2 (Dense)              (None, 256)               131328    
_________________________________________________________________
antirectifier_2 (Antirectifi (None, 512)               0         
_________________________________________________________________
dropout_2 (Dropout)          (None, 512)               0         
_________________________________________________________________
dense_3 (Dense)              (None, 10)                5130      
__________

In [9]:
# train the model
model.fit(x_train, y_train,
          batch_size=batch_size,
          epochs=epochs,
          verbose=1,
          validation_data=(x_test, y_test))

# next, compare with an equivalent network
# with 2x bigger Dense layers and ReLU

Train on 60000 samples, validate on 10000 samples
Epoch 1/40
60000/60000 [==============================] - 18s 295us/step - loss: 0.6088 - acc: 0.9132 - val_loss: 0.1541 - val_acc: 0.9610
Epoch 2/40
60000/60000 [==============================] - 17s 290us/step - loss: 0.1280 - acc: 0.9650 - val_loss: 0.1099 - val_acc: 0.9673
Epoch 3/40
60000/60000 [==============================] - 16s 262us/step - loss: 0.0837 - acc: 0.9757 - val_loss: 0.1014 - val_acc: 0.9705
Epoch 4/40
60000/60000 [==============================] - 16s 263us/step - loss: 0.0647 - acc: 0.9806 - val_loss: 0.0686 - val_acc: 0.9787
Epoch 5/40
60000/60000 [==============================] - 16s 267us/step - loss: 0.0529 - acc: 0.9837 - val_loss: 0.0651 - val_acc: 0.9805
Epoch 6/40
60000/60000 [==============================] - 16s 262us/step - loss: 0.0434 - acc: 0.9865 - val_loss: 0.0647 - val_acc: 0.9808
Epoch 7/40
60000/60000 [==============================] - 16s 274us/step - loss: 0.0379 - acc: 0.9884 - val_loss: 0.